In [1]:
import pandas as pd
import numpy as np
# import re
# import random
import keras
import scipy
import scipy.stats
import sklearn
from sklearn import svm
# from sklearn.model_selection import train_test_split
import json
from datetime import datetime
import keras.backend as K
import tensorflow as tf

In [2]:
######################################################################################
##### DATA fetching #####
######################################################################################

# cifar_10_data = tf.keras.datasets.cifar10.load_data()
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

# data -- a 10000x3072 numpy array of uint8s. Each row of the array stores a 32x32 colour image. 
#         The first 1024 entries contain the red channel values, the next 1024 the green, and the final 1024 the blue. 
#         The image is stored in row-major order, so that the first 32 entries of the array are the red channel values of the first row of the image.
# labels -- a list of 10000 numbers in the range 0-9. The number at index i indicates the label of the ith image in the array data.

print("X_Train: ", x_train.shape)
print("Y_Train: ", y_train.shape)
print("X_Test: ", x_test.shape)
print("Y_Test: ", y_test.shape)

170500096/170498071 [==============================] - 4s 0us/step
X_Train:  (50000, 32, 32, 3)
Y_Train:  (50000, 1)
X_Test:  (10000, 32, 32, 3)
Y_Test:  (10000, 1)


# AlexNet

In [3]:
######################################################################################
##### PARAMETERS #####
######################################################################################

no_of_labels = len(np.unique(y_train))
input_shape = (x_train.shape[1],x_train.shape[2],x_train.shape[3])

learning_rate = 0.001
loss = "categorical_crossentropy"

epochs = 25
batch_size = 100
validation_split = 0.2

In [4]:
######################################################################################
##### Preparing Y data for model #####
######################################################################################

y_train_input = np.zeros(shape=(y_train.shape[0],no_of_labels))
for row in range(y_train.shape[0]):
  y_train_input[row][y_train[row][0]] = 1

y_test_input = np.zeros(shape=(y_test.shape[0],no_of_labels))
for row in range(y_test.shape[0]):
  y_test_input[row][y_test[row][0]] = 1

In [7]:
######################################################################################
##### Initializing MODEL #####
######################################################################################

model = keras.models.Sequential()

model.add(keras.layers.Conv2D(96, (5,5), strides = (2,2), activation = 'relu', input_shape = input_shape))
# model.add(keras.layers.BatchNormalization(trainable = True))
model.add(keras.layers.MaxPooling2D(pool_size=(2,2), strides = (2,2)))

model.add(keras.layers.Conv2D(256, (3,3), strides = (1,1), activation = 'relu', padding = "same"))
# model.add(keras.layers.BatchNormalization(trainable = True))
model.add(keras.layers.MaxPooling2D(pool_size=(2,2), strides = (1,1)))

model.add(keras.layers.Conv2D(384, (2,2), strides = (1,1), activation = 'relu', padding = "same"))
# model.add(keras.layers.BatchNormalization(trainable = True))

model.add(keras.layers.Conv2D(384, (2,2), strides = (1,1), activation = 'relu', padding = "same"))
# model.add(keras.layers.BatchNormalization(trainable = True))

model.add(keras.layers.Conv2D(256, (2,2), strides=(1,1), activation = 'relu', padding = "same"))
# model.add(keras.layers.BatchNormalization(trainable = True))
model.add(keras.layers.MaxPooling2D(pool_size=(2,2), strides = (1,1)))

model.add(keras.layers.Flatten())

model.add(keras.layers.Dense(256, activation = 'relu'))
model.add(keras.layers.Dense(256, activation = 'relu'))
model.add(keras.layers.Dense(no_of_labels, activation = 'softmax'))

model.compile(optimizer = keras.optimizers.Adam(learning_rate = learning_rate), loss = loss, metrics = ['BinaryAccuracy'])

model.build()
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 14, 14, 96)        7296      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 7, 7, 96)          0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 7, 7, 256)         221440    
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 6, 6, 256)         0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 6, 6, 384)         393600    
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 6, 6, 384)         590208    
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 6, 6, 256)        

In [8]:
######################################################################################
##### Training MODEL #####
######################################################################################

print("Training start time : "+str(datetime.now()))

## Model Fitting on Train Data
model.fit(x_train, y_train_input, epochs = epochs, validation_split = validation_split, batch_size = batch_size, verbose = 1)
# es_callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

print("Training end time : "+str(datetime.now()))

model.save("main_model.h5")

Training start time : 2020-10-24 08:17:46.008129
Epoch 1/25
400/400 [==============================] - 6s 15ms/step - loss: 1.9655 - binary_accuracy: 0.9049 - val_loss: 1.5063 - val_binary_accuracy: 0.9107
Epoch 2/25
400/400 [==============================] - 6s 15ms/step - loss: 1.3604 - binary_accuracy: 0.9175 - val_loss: 1.3182 - val_binary_accuracy: 0.9201
Epoch 3/25
400/400 [==============================] - 6s 15ms/step - loss: 1.1968 - binary_accuracy: 0.9261 - val_loss: 1.2439 - val_binary_accuracy: 0.9235
Epoch 4/25
400/400 [==============================] - 6s 16ms/step - loss: 1.0713 - binary_accuracy: 0.9330 - val_loss: 1.1108 - val_binary_accuracy: 0.9309
Epoch 5/25
400/400 [==============================] - 6s 15ms/step - loss: 0.9707 - binary_accuracy: 0.9386 - val_loss: 1.0401 - val_binary_accuracy: 0.9354
Epoch 6/25
400/400 [==============================] - 6s 15ms/step - loss: 0.8907 - binary_accuracy: 0.9432 - val_loss: 1.0492 - val_binary_accuracy: 0.9354
Epoch 7/2

In [9]:
######################################################################################
##### Generate prediction and performance metrics using Train Data (SEEN Data)
######################################################################################

train_prediction_output = model.predict(x_train)

## train data prediction mapped to class with highest output value
y_train_prediction_output = np.empty(shape=(y_train.shape[0]))
for row in range(train_prediction_output.shape[0]):
  y_train_prediction_output[row] = np.where(train_prediction_output[row] == np.amax(train_prediction_output[row]))[0][0]

## reshaping y_train to required format
y_train_reshaped = np.reshape(y_train, y_train.shape[0])

confusion_df = pd.DataFrame({'y_true':y_train_reshaped, 'y_pred':y_train_prediction_output})
confusion_df['comparison'] = (confusion_df['y_true'] == confusion_df['y_pred']).astype(int)
print("=== For Train Data ===")
print("Total accuracy: ",sum(confusion_df['comparison'])/confusion_df.shape[0])

confusion_matrix = sklearn.metrics.confusion_matrix(y_train_reshaped, y_train_prediction_output)
confusion_matrix_normalized = confusion_matrix.astype('float') / confusion_matrix.sum(axis=1)[:, np.newaxis]
print("Class wise accuracies: ")
confusion_matrix_normalized.diagonal()

=== For Train Data ===
Total accuracy:  0.87962
Class wise accuracies: 


array([0.9268, 0.9318, 0.8166, 0.7444, 0.8318, 0.8486, 0.8862, 0.956 ,
       0.9276, 0.9264])

In [10]:
######################################################################################
##### Generate prediction and performance metrics using Test Data (UNSEEN Data)
######################################################################################

test_prediction_output = model.predict(x_test)

## TEST data prediction mapped to class with highest output value
y_test_prediction_output = np.empty(shape=(y_test.shape[0]))
for row in range(test_prediction_output.shape[0]):
  y_test_prediction_output[row] = np.where(test_prediction_output[row] == np.amax(test_prediction_output[row]))[0][0]

## reshaping y_test to required format
y_test_reshaped = np.reshape(y_test, y_test.shape[0])

confusion_df = pd.DataFrame({'y_true':y_test_reshaped, 'y_pred':y_test_prediction_output})
confusion_df['comparison'] = (confusion_df['y_true'] == confusion_df['y_pred']).astype(int)
print("=== For Test Data ===")
print("Total accuracy: ",sum(confusion_df['comparison'])/confusion_df.shape[0])

confusion_matrix = sklearn.metrics.confusion_matrix(y_test_reshaped, y_test_prediction_output)
confusion_matrix_normalized = confusion_matrix.astype('float') / confusion_matrix.sum(axis=1)[:, np.newaxis]
print("Class wise accuracies: ")
confusion_matrix_normalized.diagonal()

=== For Test Data ===
Total accuracy:  0.6684
Class wise accuracies: 


array([0.772, 0.8  , 0.482, 0.405, 0.557, 0.583, 0.722, 0.796, 0.783,
       0.784])

# SVM - 1
##### Extracting model embeddings at the flattening layer

In [11]:
K.clear_session()

######################################################################################
##### Extract sub-network trained AlexNet for embedding extraction
######################################################################################

## DEFINE the model sub-structure as required
model_svm1_embedding = keras.models.Sequential()

model_svm1_embedding.add(keras.layers.Conv2D(96, (5,5), strides = (2,2), activation = 'relu', input_shape = input_shape))
# model_svm1_embedding.add(keras.layers.BatchNormalization(trainable = True))
model_svm1_embedding.add(keras.layers.MaxPooling2D(pool_size=(2,2), strides = (2,2)))

model_svm1_embedding.add(keras.layers.Conv2D(256, (3,3), strides = (1,1), activation = 'relu', padding = "same"))
# model_svm1_embedding.add(keras.layers.BatchNormalization(trainable = True))
model_svm1_embedding.add(keras.layers.MaxPooling2D(pool_size=(2,2), strides = (1,1)))

## extract weight from already trained AlexNet to compile the new partial network for embedding extraction
i = 0
for layer in model_svm1_embedding.layers:
  layer.set_weights(model.layers[i].get_weights())
  i = i+1

model_svm1_embedding.add(keras.layers.Flatten())

model_svm1_embedding.compile(optimizer = keras.optimizers.Adam(learning_rate = learning_rate), loss = loss, metrics = ['BinaryAccuracy'])

model_svm1_embedding.build()
model_svm1_embedding.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 14, 14, 96)        7296      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 7, 7, 96)          0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 7, 7, 256)         221440    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 256)         0         
_________________________________________________________________
flatten (Flatten)            (None, 9216)              0         
Total params: 228,736
Trainable params: 228,736
Non-trainable params: 0
_________________________________________________________________


In [12]:
######################################################################################
##### Generate the embeddings for the train and test feature sets
######################################################################################

x_train_svm1 = model_svm1_embedding.predict(x_train)
x_test_svm1 = model_svm1_embedding.predict(x_test)

## using y_train_reshaped and y_test_reshaped for SVM 

In [13]:
######################################################################################
##### Training SVM model 1 #####
######################################################################################

svm_model_1 = svm.LinearSVC()

## Train
svm_model_1.fit(x_train_svm1, y_train_reshaped)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [14]:
######################################################################################
##### Generate prediction and performance metrics using Train Data (SEEN Data)
######################################################################################

train_prediction_output = svm_model_1.predict(x_train_svm1)

## reshaping y_train to required format
y_train_reshaped = np.reshape(y_train, y_train.shape[0])

confusion_df = pd.DataFrame({'y_true':y_train_reshaped, 'y_pred':train_prediction_output})
confusion_df['comparison'] = (confusion_df['y_true'] == confusion_df['y_pred']).astype(int)
print("=== For Train Data ===")
print("Total accuracy: ",sum(confusion_df['comparison'])/confusion_df.shape[0])

confusion_matrix = sklearn.metrics.confusion_matrix(y_train_reshaped, train_prediction_output)
confusion_matrix_normalized = confusion_matrix.astype('float') / confusion_matrix.sum(axis=1)[:, np.newaxis]
print("Class wise accuracies: ")
confusion_matrix_normalized.diagonal()

=== For Train Data ===
Total accuracy:  0.90848
Class wise accuracies: 


array([0.9084, 0.9922, 0.8968, 0.7152, 0.8704, 0.8756, 0.9634, 0.9226,
       0.9916, 0.9486])

In [15]:
######################################################################################
##### Generate prediction and performance metrics using Test Data (SEEN Data)
######################################################################################

test_prediction_output = svm_model_1.predict(x_test_svm1)

## reshaping y_test to required format
y_test_reshaped = np.reshape(y_test, y_test.shape[0])

confusion_df = pd.DataFrame({'y_true':y_test_reshaped, 'y_pred':test_prediction_output})
confusion_df['comparison'] = (confusion_df['y_true'] == confusion_df['y_pred']).astype(int)
print("=== For test Data ===")
print("Total accuracy: ",sum(confusion_df['comparison'])/confusion_df.shape[0])

confusion_matrix = sklearn.metrics.confusion_matrix(y_test_reshaped, test_prediction_output)
confusion_matrix_normalized = confusion_matrix.astype('float') / confusion_matrix.sum(axis=1)[:, np.newaxis]
print("Class wise accuracies: ")
confusion_matrix_normalized.diagonal()

=== For test Data ===
Total accuracy:  0.5665
Class wise accuracies: 


array([0.582, 0.689, 0.52 , 0.322, 0.501, 0.484, 0.641, 0.61 , 0.706,
       0.61 ])

# SVM - 2
##### Extracting model embeddings at the final dense layer before the last dense layer which has units equal to the number of different classes

In [18]:
######################################################################################
##### Extract sub-network trained AlexNet for embedding extraction
######################################################################################

## DEFINE the model sub-structure as required
model_svm2_embedding = keras.models.Sequential()

model_svm2_embedding.add(keras.layers.Conv2D(96, (5,5), strides = (2,2), activation = 'relu', input_shape = input_shape))
# model_svm2_embedding.add(keras.layers.BatchNormalization(trainable = True))
model_svm2_embedding.add(keras.layers.MaxPooling2D(pool_size=(2,2), strides = (2,2)))

model_svm2_embedding.add(keras.layers.Conv2D(256, (3,3), strides = (1,1), activation = 'relu', padding = "same"))
# model_svm2_embedding.add(keras.layers.BatchNormalization(trainable = True))
model_svm2_embedding.add(keras.layers.MaxPooling2D(pool_size=(2,2), strides = (1,1)))

model_svm2_embedding.add(keras.layers.Conv2D(384, (2,2), strides = (1,1), activation = 'relu', padding = "same"))
# model_svm2_embedding.add(keras.layers.BatchNormalization(trainable = True))

model_svm2_embedding.add(keras.layers.Conv2D(384, (2,2), strides = (1,1), activation = 'relu', padding = "same"))
# model_svm2_embedding.add(keras.layers.BatchNormalization(trainable = True))

model_svm2_embedding.add(keras.layers.Conv2D(256, (2,2), strides=(1,1), activation = 'relu', padding = "same"))
# model_svm2_embedding.add(keras.layers.BatchNormalization(trainable = True))
model_svm2_embedding.add(keras.layers.MaxPooling2D(pool_size=(2,2), strides = (1,1)))

## extract weight from already trained AlexNet to compile the new partial network for embedding extraction
i = 0
for layer in model_svm2_embedding.layers:
  layer.set_weights(model.layers[i].get_weights())
  i = i+1

model_svm2_embedding.add(keras.layers.Flatten())

model_svm2_embedding.compile(optimizer = keras.optimizers.Adam(learning_rate = learning_rate), loss = loss, metrics = ['BinaryAccuracy'])

model_svm2_embedding.build()
model_svm2_embedding.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 14, 14, 96)        7296      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 7, 7, 96)          0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 7, 7, 256)         221440    
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 6, 6, 256)         0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 6, 6, 384)         393600    
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 6, 6, 384)         590208    
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 6, 6, 256)        

In [20]:
######################################################################################
##### Generate the embeddings for the train and test feature sets
######################################################################################

x_train_svm2 = model_svm2_embedding.predict(x_train)
x_test_svm2 = model_svm2_embedding.predict(x_test)

## using y_train_reshaped and y_test_reshaped for SVM 

In [21]:
######################################################################################
##### Training SVM model 1 #####
######################################################################################

svm_model_2 = svm.LinearSVC()

## Train
svm_model_2.fit(x_train_svm2, y_train_reshaped)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [22]:
######################################################################################
##### Generate prediction and performance metrics using Train Data (SEEN Data)
######################################################################################

train_prediction_output = svm_model_2.predict(x_train_svm2)

## reshaping y_train to required format
y_train_reshaped = np.reshape(y_train, y_train.shape[0])

confusion_df = pd.DataFrame({'y_true':y_train_reshaped, 'y_pred':train_prediction_output})
confusion_df['comparison'] = (confusion_df['y_true'] == confusion_df['y_pred']).astype(int)
print("=== For Train Data ===")
print("Total accuracy: ",sum(confusion_df['comparison'])/confusion_df.shape[0])

confusion_matrix = sklearn.metrics.confusion_matrix(y_train_reshaped, train_prediction_output)
confusion_matrix_normalized = confusion_matrix.astype('float') / confusion_matrix.sum(axis=1)[:, np.newaxis]
print("Class wise accuracies: ")
confusion_matrix_normalized.diagonal()

=== For Train Data ===
Total accuracy:  0.97222
Class wise accuracies: 


array([0.9888, 0.997 , 0.9678, 0.9278, 0.9838, 0.901 , 0.9932, 0.9746,
       0.9956, 0.9926])

In [23]:
######################################################################################
##### Generate prediction and performance metrics using Test Data (SEEN Data)
######################################################################################

test_prediction_output = svm_model_2.predict(x_test_svm2)

## reshaping y_test to required format
y_test_reshaped = np.reshape(y_test, y_test.shape[0])

confusion_df = pd.DataFrame({'y_true':y_test_reshaped, 'y_pred':test_prediction_output})
confusion_df['comparison'] = (confusion_df['y_true'] == confusion_df['y_pred']).astype(int)
print("=== For test Data ===")
print("Total accuracy: ",sum(confusion_df['comparison'])/confusion_df.shape[0])

confusion_matrix = sklearn.metrics.confusion_matrix(y_test_reshaped, test_prediction_output)
confusion_matrix_normalized = confusion_matrix.astype('float') / confusion_matrix.sum(axis=1)[:, np.newaxis]
print("Class wise accuracies: ")
confusion_matrix_normalized.diagonal()

=== For test Data ===
Total accuracy:  0.6214
Class wise accuracies: 


array([0.669, 0.746, 0.521, 0.439, 0.561, 0.464, 0.716, 0.65 , 0.752,
       0.696])